# Tabular data
In the "Further Research" section of chapter 9 of the book, the 1st assignment is to pick a competition on Kaggle based on tabular data. I chose to join the [Bike Sharing Demand](https://www.kaggle.com/c/bike-sharing-demand/overview/description). This competion was closed years ago, but the open ones are all about predicting sales and I don't want to simply replicate what was in the book.

In [1]:
#hide
!pip install -Uqq fastbook kaggle waterfallcharts treeinterpreter dtreeviz
import fastbook
fastbook.setup_book()

In [2]:
from fastbook import *
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from dtreeviz.trees import *
from IPython.display import Image, display_svg, SVG

pd.options.display.max_rows = 20
pd.options.display.max_columns = 8

Set up the Kaggle API and get the data:

In [3]:
creds = '{"username":"francescolost","key":"b1cc19c1a43064e643fa0eabbc4ebffe"}'

In [4]:
cred_path = Path('~/.kaggle/kaggle.json').expanduser()
print(cred_path)
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

/root/.kaggle/kaggle.json


In [5]:
path = URLs.path('tabularplayground')
path

Path('/root/.fastai/archive/tabularplayground')

In [6]:
Path.BASE_PATH = path

In [7]:
from kaggle import api

In [8]:
if not path.exists():
    path.mkdir(parents=True)

In [9]:
api.competition_download_cli('bike-sharing-demand', path=path)

bike-sharing-demand.zip: Skipping, found more recently modified local copy (use --force to force download)


In [10]:
path.ls()

(#6) [Path('test.csv'),Path('tabular-playground-series-feb-2021.zip'),Path('bike-sharing-demand.zip'),Path('train.csv'),Path('sampleSubmission.csv'),Path('sample_submission.csv')]

In [11]:
file_extract(path/'bike-sharing-demand.zip')

In [12]:
path.ls(file_type='text')

(#4) [Path('test.csv'),Path('train.csv'),Path('sampleSubmission.csv'),Path('sample_submission.csv')]

In [13]:
df_train = pd.read_csv(path/'train.csv', low_memory=False)
df_test = pd.read_csv(path/'test.csv', low_memory=False)

In [14]:
len(df_train.columns),len(df_test.columns)

(12, 9)

In [15]:
df_train.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'],
      dtype='object')

In [16]:
df_test.columns

Index(['datetime', 'season', 'holiday', 'workingday', 'weather', 'temp',
       'atemp', 'humidity', 'windspeed'],
      dtype='object')

In [17]:
df_train.drop(columns=['registered','casual'],inplace=True)

In [18]:
df_train.dtypes

datetime       object
season          int64
holiday         int64
workingday      int64
weather         int64
temp          float64
atemp         float64
humidity        int64
windspeed     float64
count           int64
dtype: object

Merge in a single dataframe for better handling:

In [19]:
df = pd.concat([df_train, df_test], axis=0)

In [22]:
df['is_test'] = 0
df.loc[ df['count'].isna(), 'is_test' ] = 1

Get additional info out of `datetime`:


In [20]:
add_datepart??

In [21]:
df = add_datepart(df, 'datetime', time=True)

In [23]:
for c in df.columns:
  print(c,':', len(df[c].unique()), ':', df[c].unique())

season : 4 : [1 2 3 4]
holiday : 2 : [0 1]
datetimeWeek : 52 : <IntegerArray>
[52,  1,  2,  3,  5,  6,  7,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 22,
 23, 24, 26, 27, 28, 29, 31, 32, 33, 35, 36, 37, 38, 39, 40, 41, 42, 44, 45,
 46, 48, 49, 50, 51, 12, 25, 47,  4,  8, 21, 30, 34, 43]
Length: 52, dtype: UInt32
workingday : 2 : [0 1]
weather : 4 : [1 2 3 4]
temp : 50 : [ 9.84  9.02  8.2  13.12 15.58 14.76 17.22 18.86 18.04 16.4  13.94 12.3  10.66  6.56  5.74  7.38  4.92 11.48  4.1   3.28  2.46 21.32 22.96 23.78 24.6  19.68 22.14 20.5  27.06 26.24 25.42 27.88 28.7
 30.34 31.16 29.52 33.62 35.26 36.9  32.8  31.98 34.44 36.08 37.72 38.54  1.64  0.82 39.36 41.   40.18]
atemp : 65 : [14.395 13.635 12.88  17.425 19.695 16.665 21.21  22.725 21.97  20.455 11.365 10.605  9.85   8.335  6.82   5.305  6.06   9.09  12.12   7.575 15.91   3.03   3.79   4.545 15.15  18.18  25.    26.515
 27.275 29.545 23.485 25.76  31.06  30.305 24.24  18.94  31.82  32.575 33.335 28.79  34.85  35.605 37.12  40.15  4

In [25]:
df.drop(columns=['datetimeMinute','datetimeSecond'], inplace=True)

Create a validation set here: use the first 13 days of the month as training set and the days from 15th to 20th as validation set. The days from the 21st to the end of the month are already in the test set.

In [26]:
df['is_valid'] = 0
df.loc[ df['datetimeDay'] > 13, 'is_valid' ] = 1

In [27]:
df = df.astype({'datetimeWeek' : 'uint32'})

Probably it's better to treat `season` and `weather` as ordinal categories:

In [28]:
df['season'] = df['season'].astype('category')
df['season'].cat.set_categories([1,2,3,4], ordered=True, inplace=True)

In [29]:
df['weather'] = df['weather'].astype('category')
df['weather'].cat.set_categories([1,2,3,4], ordered=True, inplace=True)

The model is evaluated on the RMSLE:
$ \sqrt{\frac{1}{n} \sum_{i=1}^n (\log(p_i + 1) - \log(a_i+1))^2 } $

In [30]:
targetVar = 'count'

In [33]:
df[ df['is_test']==0 ][targetVar].min(),df[ df['is_test']==0 ][targetVar].max()

(1.0, 977.0)

In [34]:
preds = np.random.random_integers(1,1000,len(df[df['is_valid']==1]))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: This function is deprecated. Please call randint(1, 1000 + 1) instead
  """Entry point for launching an IPython kernel.


In [35]:
L(list(preds))

(#9927) [971,111,338,123,705,447,611,291,671,412...]

In [36]:
def rmsle(p,a):
  return np.sqrt( ((np.log(p+1) - np.log(a+1))**2).mean() )

In [37]:
def m_rmsle(m, xs, y): return rmsle(m.predict(xs), y)

In [38]:
rmsle(preds, df[ df['is_valid']==1 ][targetVar])

2.125231281916073

Build a `TabularPandas` object:

In [48]:
df_train = df[ df['is_test']==0 ]

cond = df_train['is_valid']==0
train_idx = np.where( cond)[0]
valid_idx = np.where(~cond)[0]

splits = (list(train_idx),list(valid_idx))

In [49]:
[ (c,df[c].dtype) for c in df.columns ]

[('season', CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)),
 ('holiday', dtype('int64')),
 ('datetimeWeek', dtype('uint32')),
 ('workingday', dtype('int64')),
 ('weather', CategoricalDtype(categories=[1, 2, 3, 4], ordered=True)),
 ('temp', dtype('float64')),
 ('atemp', dtype('float64')),
 ('humidity', dtype('int64')),
 ('windspeed', dtype('float64')),
 ('count', dtype('float64')),
 ('datetimeYear', dtype('int64')),
 ('datetimeMonth', dtype('int64')),
 ('datetimeDay', dtype('int64')),
 ('datetimeDayofweek', dtype('int64')),
 ('datetimeDayofyear', dtype('int64')),
 ('datetimeIs_month_end', dtype('bool')),
 ('datetimeIs_month_start', dtype('bool')),
 ('datetimeIs_quarter_end', dtype('bool')),
 ('datetimeIs_quarter_start', dtype('bool')),
 ('datetimeIs_year_end', dtype('bool')),
 ('datetimeIs_year_start', dtype('bool')),
 ('datetimeHour', dtype('int64')),
 ('datetimeElapsed', dtype('O')),
 ('is_test', dtype('int64')),
 ('is_valid', dtype('int64'))]

In [50]:
def cont_cat_split(df, max_card=20, dep_var=None):
    cont_names, cat_names = [], []
    for label in df:
        if label in L(dep_var): continue
        
        # mod to detect ProductSize type properly
        if (df[label].dtype.name == 'category'):
          cat_names.append(label)
          continue

        if (np.issubdtype(df[label].dtype, np.integer) and
            df[label].unique().shape[0] > max_card or
            np.issubdtype(df[label].dtype, np.floating)):
            cont_names.append(label)
        else: cat_names.append(label)
    return cont_names, cat_names

In [51]:
cont,cat = cont_cat_split(df, max_card=100, dep_var=targetVar)

`Categorify` is not used as there are no string-typed columns

In [52]:
procs = [FillMissing]

In [53]:
to = TabularPandas(df_train, procs=procs, cat_names=cat, cont_names=cont, y_names=targetVar, splits=splits)

In [54]:
to.show(5)

,season,holiday,datetimeWeek,workingday,weather,humidity,datetimeYear,datetimeMonth,datetimeDay,datetimeDayofweek,datetimeIs_month_end,datetimeIs_month_start,datetimeIs_quarter_end,datetimeIs_quarter_start,datetimeIs_year_end,datetimeIs_year_start,datetimeHour,datetimeElapsed,is_test,is_valid,temp,atemp,windspeed,datetimeDayofyear,count
0,1,0,52,0,1,81,2011,1,1,5,False,True,False,True,False,True,0,1293840000,0,0,9.84,14.395,0.0,1,16.0
1,1,0,52,0,1,80,2011,1,1,5,False,True,False,True,False,True,1,1293843600,0,0,9.02,13.635,0.0,1,40.0
2,1,0,52,0,1,80,2011,1,1,5,False,True,False,True,False,True,2,1293847200,0,0,9.02,13.635,0.0,1,32.0
3,1,0,52,0,1,75,2011,1,1,5,False,True,False,True,False,True,3,1293850800,0,0,9.84,14.395,0.0,1,13.0
4,1,0,52,0,1,75,2011,1,1,5,False,True,False,True,False,True,4,1293854400,0,0,9.84,14.395,0.0,1,1.0


In [55]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y

Use a function to quickly build a Random Forest:

In [59]:
def rf(xs,y,n_estimators=40,max_samples=5000,max_features=0.5,min_samples_leaf=10,**kwargs):
  return RandomForestRegressor(n_jobs=-1,
                               n_estimators=n_estimators,max_samples=max_samples,max_features=max_features,
                               min_samples_leaf=min_samples_leaf,oob_score=True).fit(xs,y)

In [60]:
m = rf(xs,y)

In [61]:
m_rmsle(m, xs, y),m_rmsle(m, valid_xs, valid_y)

(0.4517481229492313, 0.4861685296972013)